In [115]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
import matplotlib.pyplot as plt
import seaborn as sns


In [116]:
data = pd.read_csv("data/Custom_Crops_yield_Historical_Dataset.csv")

In [117]:
data

,Dist Code,Year,State Code,State Name,Dist Name,Crop,Area_ha,Yield_kg_per_ha,N_req_kg_per_ha,P_req_kg_per_ha,K_req_kg_per_ha,Total_N_kg,Total_P_kg,Total_K_kg,Temperature_C,Humidity_%,pH,Rainfall_mm,Wind_Speed_m_s,Solar_Radiation_MJ_m2_day
0,1,1966,14,Chhattisgarh,Durg,rice,548000.0,337.59,8.43975,4.05108,7.42698,4.624983e+06,2.219992e+06,4.069985e+06,25,80,6.5,1200,2.0,18
1,1,1966,14,Chhattisgarh,Durg,maize,3000.0,666.67,18.00009,8.00004,11.33339,5.400027e+04,2.400012e+04,3.400017e+04,22,70,6.0,800,2.5,20
2,1,1966,14,Chhattisgarh,Durg,chickpea,54000.0,500.00,9.00000,5.00000,9.00000,4.860000e+05,2.700000e+05,4.860000e+05,20,60,6.5,600,1.5,16
3,1,1967,14,Chhattisgarh,Durg,rice,547000.0,747.71,18.69275,8.97252,16.44962,1.022493e+07,4.907968e+06,8.997942e+06,25,80,6.5,1200,2.0,18
4,1,1967,14,Chhattisgarh,Durg,maize,3000.0,1000.00,27.00000,12.00000,17.00000,8.100000e+04,3.600000e+04,5.100000e+04,22,70,6.0,800,2.5,20
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
50760,917,2016,15,Jharkhand,Singhbhum,maize,750.0,1869.97,50.48919,22.43964,31.78949,3.786689e+04,1.682973e+04,2.384212e+04,22,70,6.0,800,2.5,20
50761,917,2016,15,Jharkhand,Singhbhum,chickpea,150.0,1171.23,21.08214,11.71230,21.08214,3.162321e+03,1.756845e+03,3.162321e+03,20,60,6.5,600,1.5,16
50762,917,2017,15,Jharkhand,Singhbhum,rice,386910.0,1731.62,43.29050,20.77944,38.09564,1.674953e+07,8.039773e+06,1.473958e+07,25,80,6.5,1200,2.0,18
50763,917,2017,15,Jharkhand,Singhbhum,maize,27070.0,1507.72,40.70844,18.09264,25.63124,1.101977e+06,4.897678e+05,6.938377e+05,22,70,6.0,800,2.5,20


In [118]:
data = data.drop([ 'N_req_kg_per_ha', 'P_req_kg_per_ha', 'K_req_kg_per_ha',"Total_N_kg","Total_P_kg","Total_K_kg","Wind_Speed_m_s","Solar_Radiation_MJ_m2_day"], axis=1)

In [119]:
X = data.drop('Yield_kg_per_ha', axis=1)
y = data['Yield_kg_per_ha']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
# ...existing code...
# ...existing code...
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OrdinalEncoder, StandardScaler
from sklearn.pipeline import Pipeline
import joblib
import os
import pandas as pd

# define categorical columns (replace if your column names differ)
categorical_cols = ['State Name', 'Dist Name', 'Crop']

# compute numeric columns from X (assumes X already defined)
numeric_cols = [c for c in X.columns if c not in categorical_cols]

# build preprocessor (use OrdinalEncoder for categorical ordinal encoding)
preprocessor = ColumnTransformer(
    transformers=[
        ('num', StandardScaler(), numeric_cols),
        ('cat', OrdinalEncoder(handle_unknown='use_encoded_value', unknown_value=-1), categorical_cols),
    ],
    remainder='drop'
)

# lightweight pipeline
base_pipeline = Pipeline(steps=[('preprocessor', preprocessor)])

# wrapper that returns a pandas DataFrame with cleaned column names
class PandasOutputPreprocessor:
    def __init__(self, pipeline):
        self.pipeline = pipeline

    def fit(self, X, y=None):
        self.pipeline.fit(X, y)
        # build cleaned feature names once fitted
        try:
            raw_names = self.pipeline.named_steps['preprocessor'].get_feature_names_out()
        except Exception:
            # fallback: use numeric + categorical (in order)
            raw_names = numeric_cols + categorical_cols
        # remove transformer prefixes like 'num__' or 'cat__'
        self.feature_names_ = [n.split('__', 1)[1] if '__' in n else n for n in raw_names]
        return self

    def transform(self, X):
        arr = self.pipeline.transform(X)
        df = pd.DataFrame(arr, columns=self.feature_names_, index=getattr(X, "index", None))
        return df

    def fit_transform(self, X, y=None):
        self.fit(X, y)
        return self.transform(X)

    def get_feature_names_out(self):
        return list(self.feature_names_)

# instantiate, fit and save the pandas-output wrapper
preprocessor_pipeline = PandasOutputPreprocessor(base_pipeline)
preprocessor_pipeline.fit(X_train)

os.makedirs('models', exist_ok=True)
joblib.dump(preprocessor_pipeline, 'models/preprocessor.joblib')

print("Preprocessor saved to models/preprocessor.joblib")
# ...existing code...

Preprocessor saved to models/preprocessor.joblib


In [120]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
X_train = preprocessor_pipeline.transform(X_train)
X_test = preprocessor_pipeline.transform(X_test)

In [121]:
data

,Dist Code,Year,State Code,State Name,Dist Name,Crop,Area_ha,Yield_kg_per_ha,Temperature_C,Humidity_%,pH,Rainfall_mm
0,1,1966,14,Chhattisgarh,Durg,rice,548000.0,337.59,25,80,6.5,1200
1,1,1966,14,Chhattisgarh,Durg,maize,3000.0,666.67,22,70,6.0,800
2,1,1966,14,Chhattisgarh,Durg,chickpea,54000.0,500.00,20,60,6.5,600
3,1,1967,14,Chhattisgarh,Durg,rice,547000.0,747.71,25,80,6.5,1200
4,1,1967,14,Chhattisgarh,Durg,maize,3000.0,1000.00,22,70,6.0,800
...,...,...,...,...,...,...,...,...,...,...,...,...
50760,917,2016,15,Jharkhand,Singhbhum,maize,750.0,1869.97,22,70,6.0,800
50761,917,2016,15,Jharkhand,Singhbhum,chickpea,150.0,1171.23,20,60,6.5,600
50762,917,2017,15,Jharkhand,Singhbhum,rice,386910.0,1731.62,25,80,6.5,1200
50763,917,2017,15,Jharkhand,Singhbhum,maize,27070.0,1507.72,22,70,6.0,800


In [122]:
pd.DataFrame(X_train)

,Dist Code,Year,State Code,Area_ha,Temperature_C,Humidity_%,pH,Rainfall_mm,State Name,Dist Name,Crop
33299,-0.194765,-0.710813,0.582036,-0.561472,1.709719,-0.575607,-1.145124,-0.620114,17.0,95.0,1.0
911,-0.908291,0.225962,0.993566,2.638044,0.626809,1.356234,0.873268,1.467980,3.0,283.0,3.0
2772,-0.863696,-1.246114,-0.652553,-0.553509,-0.456101,0.068340,-1.145124,-0.202496,10.0,243.0,2.0
41078,1.321479,1.162738,1.610860,0.553315,0.626809,1.356234,0.873268,1.467980,1.0,151.0,3.0
2351,-0.874845,-0.175513,-0.652553,-0.299565,0.626809,1.356234,0.873268,1.467980,10.0,292.0,3.0
...,...,...,...,...,...,...,...,...,...,...,...
11284,-0.689030,-0.643901,-0.858318,-0.432288,-0.456101,0.068340,-1.145124,-0.202496,8.0,162.0,2.0
44732,1.693108,-0.309338,-0.241023,-0.393533,-0.456101,0.068340,-1.145124,-0.202496,12.0,144.0,2.0
38158,-0.075844,0.828175,0.582036,-0.471043,-0.456101,0.068340,-1.145124,-0.202496,17.0,159.0,2.0
860,-0.908291,-0.911551,0.993566,2.192979,0.626809,1.356234,0.873268,1.467980,3.0,283.0,3.0


In [123]:
from xgboost import XGBRegressor
from sklearn.metrics import mean_squared_error
model = XGBRegressor()
model.fit(X_train, y_train)


XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=None, device=None, early_stopping_rounds=None,
             enable_categorical=False, eval_metric=None, feature_types=None,
             feature_weights=None, gamma=None, grow_policy=None,
             importance_type=None, interaction_constraints=None,
             learning_rate=None, max_bin=None, max_cat_threshold=None,
             max_cat_to_onehot=None, max_delta_step=None, max_depth=None,
             max_leaves=None, min_child_weight=None, missing=nan,
             monotone_constraints=None, multi_strategy=None, n_estimators=None,
             n_jobs=None, num_parallel_tree=None, ...)

In [125]:
from sklearn.metrics import mean_squared_error, mean_absolute_error
y_pred = model.predict(X_test)
rmse = mean_squared_error(y_test, y_pred)
print(f"XGBoost Model RMSE: {rmse:.2f}")
mae = mean_absolute_error(y_test, y_pred)
print(f"XGBoost Model MAE: {mae:.2f}")

XGBoost Model RMSE: 144188.77
XGBoost Model MAE: 228.78


In [126]:
# ...existing code...
from xgboost import XGBRegressor
from sklearn.metrics import mean_squared_error
import joblib
import os

model = XGBRegressor()
model.fit(X_train, y_train)

os.makedirs('models', exist_ok=True)
# save with joblib (easy to load back in Python)
joblib.dump(model, 'models/xgb_model.joblib')
# also save in XGBoost native JSON format (optional)
model.save_model('models/xgb_model.json')

print("XGBoost model saved to models/xgb_model.joblib and models/xgb_model.json")
# ...existing code...

XGBoost model saved to models/xgb_model.joblib and models/xgb_model.json


In [127]:
# ...existing code...
import joblib
import pickle
import os

os.makedirs('models', exist_ok=True)

# save with joblib (already present) and also export .pkl for compatibility
joblib.dump(preprocessor_pipeline, 'models/preprocessor.joblib')
joblib.dump(model, 'models/xgb_model.joblib')

# explicit .pkl via joblib (acceptable) and via pickle (standard .pkl)
joblib.dump(preprocessor_pipeline, 'models/preprocessor.pkl')
joblib.dump(model, 'models/xgb_model.pkl')

with open('models/preprocessor_pickle.pkl', 'wb') as f:
    pickle.dump(preprocessor_pipeline, f)

with open('models/xgb_model_pickle.pkl', 'wb') as f:
    pickle.dump(model, f)

# optional: keep xgboost native format
model.save_model('models/xgb_model.json')

print("Saved: models/preprocessor.pkl, models/xgb_model.pkl (and joblib/json variants).")
# ...existing code...

Saved: models/preprocessor.pkl, models/xgb_model.pkl (and joblib/json variants).
